In [1]:
!pip install torch

In [2]:
!pip install idx2numpy

In [3]:
!pip install sklearn

In [4]:
!pip install pyreadr


In [1]:
# Define random state
random_state = 0

# Import libraries, modules, and functions
import time
import numpy as np
import matplotlib.pyplot as plt

# PyTorch imports
import torch
import torch.nn as nn
import idx2numpy
from sklearn.model_selection import KFold

In [2]:
# Display GPU/CUDA availability
print('\nCUDA availability:  %s\n' % (torch.cuda.is_available()))



CUDA availability:  False



In [3]:
data_path='/Users/ali/Desktop/Jul/apoe2_paper/vertex/age_cat/' # data path after downloading from mnist website abnd unziping in a folder

In [55]:
import pyreadr
image=pyreadr.read_r(data_path + 'image_all.rda')
#print(image.keys())
image = image["image_all"]
#print(image.shape)
age_cat=pyreadr.read_r(data_path + 'response.rda')
#print(sex.keys())
age_cat= age_cat["response"]
#print(sex.shape)
#print(sex)
age_cat=age_cat["age_cat"]
print(age_cat)

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    0.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
32    0.0
Name: age_cat, dtype: float64


In [56]:
age_cat[age_cat==0]=0;
age_cat[age_cat==1]=1;
type(age_cat)
age_cat

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    0.0
22    1.0
23    1.0
24    1.0
25    1.0
26    1.0
27    1.0
28    1.0
29    1.0
30    1.0
31    1.0
32    0.0
Name: age_cat, dtype: float64

In [57]:
X_raw_train_npy = image #
y_train_npy = age_cat

In [58]:
#flatten images
X_raw_train_npy =  np.reshape(X_raw_train_npy, newshape =(X_raw_train_npy.shape[0],-1))

In [59]:
print(X_raw_train_npy.shape)
print(X_raw_train_npy.shape[0])
print(y_train_npy.shape)


(33, 49603)
33
(33,)


### store original data

In [60]:
X_raw_train_npy=X_raw_train_npy.to_numpy()
y_train_npy=y_train_npy.to_numpy()

In [61]:
X_raw_train_npy_orig =  X_raw_train_npy
y_train_npy_orig = y_train_npy

In [62]:
y_train_npy = np.array(y_train_npy, dtype=int)
y_train_npy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [63]:
y_train_npy_orig.shape 

type(y_train_npy_orig)

numpy.ndarray

In [64]:
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
    print( type(X_raw_train_npy_orig[test].shape))
    

<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [65]:
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
	print('train: %s, test: %s' % (X_raw_train_npy_orig[train].shape, X_raw_train_npy_orig[test].shape))
	print('train: %s, test: %s' % (y_train_npy_orig[train].shape, y_train_npy_orig[test].shape))

train: (26, 49603), test: (7, 49603)
train: (26,), test: (7,)
train: (26, 49603), test: (7, 49603)
train: (26,), test: (7,)
train: (26, 49603), test: (7, 49603)
train: (26,), test: (7,)
train: (27, 49603), test: (6, 49603)
train: (27,), test: (6,)
train: (27, 49603), test: (6, 49603)
train: (27,), test: (6,)


In [66]:
y_train_npy_orig[train]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])

In [67]:
# MNIST PyTorch dataset class definition
class MNISTDataset(torch.utils.data.Dataset):
    """
    PyTorch dataset object for the MNIST dataset.
    
    Parameters
    ----------
    X : torch.Tensor
        Flattened image data of size (n_samples x 784).
    y : torch.Tensor
        Image labels, integer values 0, 1, 2, ..., 9.
    
    Attributes
    ----------
    X : torch.Tensor
        Flattened image data of size (n_samples x 784).
    y : torch.Tensor
        Image labels, integer values 0, 1, 2, ..., 9.
    len : int
        Length of dataset/number of observations in dataset.
    """
    
    # MNIST PyTorch dataset instantiation method
    def __init__(self, X, y):
        """
        MNIST dataset instantiation method.
        
        Parameters
        ----------
        X : torch.Tensor
            Flattened image data of size (n_samples x 784).
        y : torch.Tensor
            Image labels, integer values 0, 1, 2, ..., 9.
        
        Attributes
        ----------
        X : torch.Tensor
            Flattened image data of size (n_samples x 784).
        y : torch.Tensor
            Image labels, integer values 0, 1, 2, ..., 9.
        len : int
            Length of dataset/number of observations in dataset.
        """
        
        # Assign attributes
        self.X = X         # flattened image data
        self.y = y         # image labels
        self.len = len(X)  # length of dataset

    # Dataset length method
    def __len__(self):
        """
        Method that returns length of dataset/number of observations in dataset.
        
        Returns
        -------
        self.len : int
            Length of dataset/number of observations in dataset.
        """
        
        return self.len

    # Dataset indexing method
    def __getitem__(self, idx):
        """
        Method that retrieves samples and corresponding labels from the dataset.
        
        Parameters
        ----------
        idx : int
            Integer index of dataset sample to be retrieved.
        
        Returns
        -------
        X_samp : torch.Tensor
            Image sample.
        y_samp : torch.Tensor
            Label corresponding to image sample.
        """
        
        # Retrieve sample of data indexed by idx
        X_samp = self.X[idx, :]
        y_samp = self.y[idx]

        # Return data sample
        return X_samp, y_samp


In [68]:
# Logistic regression model class definition
class LogisticRegression(nn.Module):
    """
    PyTorch implementation of a logistic regression model.
    
    Parameters
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.

    Attributes
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.
    linear : torch.nn.Linear
        Fully-connected/linear layer.
    """
    
    # Logistic regression instantiation method
    def __init__(self, input_dim, output_dim):
        """
        Logistic regression model instantiation method.
        
        Parameters
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.

        Attributes
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.
        linear : torch.nn.Linear
            Fully-connected/linear layer.
        """
        
        # Inherit from torch.nn.Module
        super(LogisticRegression, self).__init__()
        
        # Assign logistic regression parameters to model attributes
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.linear = nn.Linear(
            in_features=input_dim,
            out_features=output_dim,
            bias=True)
    
    # Logistic regression forward pass method
    def forward(self, x):
        """
        Logistic regression forward pass method.
        
        Parameters
        ----------
        x : torch.Tensor()
            Tensor of input data.
        
        Returns
        -------
        logits : torch.Tensor()
             Raw model predictions (not passed through sigmoid or softmax function).
        """
        
        # Calculate model predictions/logits
        logits = self.linear(x)
        
        # Return model output
        return logits


In [69]:
# Instantiate cross-entropy loss function
loss_func = nn.CrossEntropyLoss()


In [70]:
# Number of training epochs (epochs = # of passes through data)
n_epochs = 100

In [71]:
import random
torch.manual_seed(0)
np.random.seed(0)

In [72]:

testkfold=[]
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
#	print('train: %s, test: %s' % (X_raw_train_npy_orig[train].shape, X_raw_train_npy_orig[test].shape))
    X_raw_train=X_raw_train_npy_orig[train]
    X_raw_test=X_raw_train_npy_orig[test]
    y_train=np.array(y_train_npy_orig[train] , dtype=float)
    y_test=np.array(y_train_npy_orig[test], dtype=float)
    #y_train = torch.tensor(y_train, dtype=torch.long)
    #y_test = torch.tensor(y_test, dtype=torch.long)


    
    
    X_raw_train = torch.from_numpy(X_raw_train) #format torch data
    y_train = torch.from_numpy(y_train)
    X_raw_test = torch.from_numpy(X_raw_test)
    y_test = torch.from_numpy(y_test)

    X_raw_train = X_raw_train.float() #float
    X_raw_test = X_raw_test.float()
    
    #X_train = X_raw_train / 255.0 #scale
    #X_test = X_raw_test / 255.0
    X_train=X_raw_train
    X_test=X_raw_test
    
    
    train_dataset = MNISTDataset(X_train, y_train) #pytorch data
    test_dataset = MNISTDataset(X_test, y_test)
    
    batch_size = 10
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    input_dim = X_train.shape[1]             #Define model input and output dimensions from data
    output_dim = len(np.unique(y_train.numpy()))  

    
    lr_model = LogisticRegression(input_dim=input_dim, output_dim=output_dim)# Instantiate logistic regression model
    
    lr = 0.1# Instantiate stochastic gradient descent (SGD) optimizer
    
    optimizer = torch.optim.SGD(lr_model.parameters(), lr=lr)
    #optimizer = torch.optim.Adam(lr_model.parameters())
    
    print('\nTraining logistic regression model...\n')# Time model training
    time_start = time.time()
    
    lr_model.train()# train() method affects operations such as dropout and batch normalization
        # Train model/iterate over epochs
    for epoch in range(n_epochs):
        # Initialize epoch metrics variables
        n_obs = 0
        loss_sum = 0
        n_correct = 0

        # Iterate through training data mini-batchesbatches
        for images_batch, labels_batch in train_loader:
            # Zero out the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = lr_model(images_batch)         # model predictions
            loss = loss_func(y_pred, labels_batch.long())  # loss function evaluation

            # Backward pass
            loss.backward()    # backpropagation
            optimizer.step()   # update parameters according to learning rate, gradients

            # Update epoch metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            loss_sum += n_batch * loss.item()
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

        # Calculate epoch training loss and training accuracy
        loss_train = loss_sum / n_obs
        acc_train = n_correct / n_obs

        # Display training progress
        prog_disp_freq = 1   # frequency of training progress display
        if (epoch + 1) % prog_disp_freq == 0 or epoch == 0 or (epoch + 1) == n_epochs:
            print('  E%02d | train loss: %s | train acc.: %s' % 
                  (epoch + 1, '{:.4f}'.format(loss_train), '{:.4f}'.format(acc_train)))

    # Time model training
    time_end = time.time()
    print('\nLogistic regression model training complete.\n')
    print('Time to train logistic regression model:  %.1f s\n' % (time_end - time_start))
        # Place model in evaluation mode
    # .eval() method affects operations such as dropout and batch normalization
    lr_model.eval()

    # Initialize test set metrics variables
    n_obs = 0
    n_correct = 0

    # Disable gradient calculation
    with torch.no_grad():
        # Iterate through test data mini-batches
        for images_batch, labels_batch in test_loader:
            # Forward pass
            y_pred = lr_model(images_batch)  # model predictions

            # Update test set metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

    # Calculate test accuracy
    acc_test = n_correct / n_obs

    # Display test accuracy
    print('\nLogistic regression model MNIST test acc.:  %.4f\n' % (acc_test))
    testkfold.append(acc_test)



Training logistic regression model...

  E01 | train loss: 6.5717 | train acc.: 0.6154
  E02 | train loss: 7.3870 | train acc.: 0.8846
  E03 | train loss: 0.0000 | train acc.: 1.0000
  E04 | train loss: 0.0000 | train acc.: 1.0000
  E05 | train loss: 0.0000 | train acc.: 1.0000
  E06 | train loss: 0.0000 | train acc.: 1.0000
  E07 | train loss: 0.0000 | train acc.: 1.0000
  E08 | train loss: 0.0000 | train acc.: 1.0000
  E09 | train loss: 0.0000 | train acc.: 1.0000
  E10 | train loss: 0.0000 | train acc.: 1.0000
  E11 | train loss: 0.0000 | train acc.: 1.0000
  E12 | train loss: 0.0000 | train acc.: 1.0000
  E13 | train loss: 0.0000 | train acc.: 1.0000
  E14 | train loss: 0.0000 | train acc.: 1.0000
  E15 | train loss: 0.0000 | train acc.: 1.0000
  E16 | train loss: 0.0000 | train acc.: 1.0000
  E17 | train loss: 0.0000 | train acc.: 1.0000
  E18 | train loss: 0.0000 | train acc.: 1.0000
  E19 | train loss: 0.0000 | train acc.: 1.0000
  E20 | train loss: 0.0000 | train acc.: 1.0000


  E96 | train loss: 0.0000 | train acc.: 1.0000
  E97 | train loss: 0.0000 | train acc.: 1.0000
  E98 | train loss: 0.0000 | train acc.: 1.0000
  E99 | train loss: 0.0000 | train acc.: 1.0000
  E100 | train loss: 0.0000 | train acc.: 1.0000

Logistic regression model training complete.

Time to train logistic regression model:  0.3 s


Logistic regression model MNIST test acc.:  0.5714


Training logistic regression model...

  E01 | train loss: 11.1961 | train acc.: 0.7308
  E02 | train loss: 9.5207 | train acc.: 0.8846
  E03 | train loss: 8.3357 | train acc.: 0.9231
  E04 | train loss: 0.0001 | train acc.: 1.0000
  E05 | train loss: 0.0000 | train acc.: 1.0000
  E06 | train loss: 0.0000 | train acc.: 1.0000
  E07 | train loss: 0.0000 | train acc.: 1.0000
  E08 | train loss: 0.0000 | train acc.: 1.0000
  E09 | train loss: 0.0000 | train acc.: 1.0000
  E10 | train loss: 0.0000 | train acc.: 1.0000
  E11 | train loss: 0.0000 | train acc.: 1.0000
  E12 | train loss: 0.0000 | train acc.: 

  E63 | train loss: 0.0000 | train acc.: 1.0000
  E64 | train loss: 0.0000 | train acc.: 1.0000
  E65 | train loss: 0.0000 | train acc.: 1.0000
  E66 | train loss: 0.0000 | train acc.: 1.0000
  E67 | train loss: 0.0000 | train acc.: 1.0000
  E68 | train loss: 0.0000 | train acc.: 1.0000
  E69 | train loss: 0.0000 | train acc.: 1.0000
  E70 | train loss: 0.0000 | train acc.: 1.0000
  E71 | train loss: 0.0000 | train acc.: 1.0000
  E72 | train loss: 0.0000 | train acc.: 1.0000
  E73 | train loss: 0.0000 | train acc.: 1.0000
  E74 | train loss: 0.0000 | train acc.: 1.0000
  E75 | train loss: 0.0000 | train acc.: 1.0000
  E76 | train loss: 0.0000 | train acc.: 1.0000
  E77 | train loss: 0.0000 | train acc.: 1.0000
  E78 | train loss: 0.0000 | train acc.: 1.0000
  E79 | train loss: 0.0000 | train acc.: 1.0000
  E80 | train loss: 0.0000 | train acc.: 1.0000
  E81 | train loss: 0.0000 | train acc.: 1.0000
  E82 | train loss: 0.0000 | train acc.: 1.0000
  E83 | train loss: 0.0000 | train acc.:

In [73]:
np.mean(testkfold)

0.6047619047619046

In [74]:
testkfold

[0.8571428571428571,
 0.5714285714285714,
 0.42857142857142855,
 0.5,
 0.6666666666666666]

In [75]:
# Multi-layer perceptron model class definition
class MLP(nn.Module):
    """
    PyTorch implementation of a multi-layer perceptron model.
    
    Parameters
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.
    hidden_dim : int
        Size/dimensionality of the hidden layer.
    
    Attributes
    ----------
    input_dim : int
        Size/dimensionality of the model input data.
    output_dim : int
        Size/dimensionality of the model output.
    hidden_dim : int
        Size/dimensionality of the hidden layer.
    linear_1 : torch.nn.Linear
        Fully-connected/linear neural network layer.
    linear_2 : torch.nn.Linear
        Fully-connected/linear neural network layer.
    relu : torch.nn.ReLU
        Rectified linear unit activation function.
    """
    
    # MLP instantiation method
    def __init__(self, input_dim, output_dim, hidden_dim=50):
        """
        Model instantiation method.
        
        Parameters
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.
        hidden_dim : int
            Size/dimensionality of the hidden layer.

        Attributes
        ----------
        input_dim : int
            Size/dimensionality of the model input data.
        output_dim : int
            Size/dimensionality of the model output.
        hidden_dim : int
            Size/dimensionality of the hidden layer.
        linear_1 : torch.nn.Linear
            Fully-connected/linear neural network layer.
        linear_2 : torch.nn.Linear
            Fully-connected/linear neural network layer.
        relu : torch.nn.ReLU
            Rectified linear unit activation function.
        """
        
        # Inherit from torch.nn.Module
        super(MLP, self).__init__()
        
        # Assign MLP parameters to model attributes
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        # Define MLP layers
        self.linear_1 = nn.Linear(
            in_features=input_dim,
            out_features=hidden_dim,
            bias=True)
        self.linear_2 = nn.Linear(
            in_features=hidden_dim,
            out_features=hidden_dim,
            bias=True)
        self.linear_3 = nn.Linear(
            in_features=hidden_dim,
            out_features=output_dim,
            bias=True)
        
        # Define MLP activation function
        self.relu = nn.LeakyReLU()
    
    # MLP forward pass method
    def forward(self, x):
        """
        MLP forward pass method.
        
        Parameters
        ----------
        x : torch.Tensor()
            Tensor of input data.
        
        Returns
        -------
        logits : torch.Tensor()
             Raw model predictions (not passed through sigmoid or softmax function).
        """
        
        # Forward pass through model
        x = self.linear_1(x)       # first fully-connected layer
        x = self.relu(x)           # non-linear activation transformation
        
        s=self.linear_2(x)
        s = self.relu(s)           # non-linear activation transformation

        mids2=self.linear_2(s)
        mids2 = self.relu(mids2)           # non-linear activation transformation
        
        mids3=self.linear_2(mids2)
        mids3 = self.relu(mids3)           # non-linear activation transformation

        
        mids4=self.linear_2(mids3)
        mids4 = self.relu(mids4)
        
        mids5=self.linear_2(mids4)
        mids5 = self.relu(mids5)
        
        
        logits = self.linear_3(s)  # second fully-connected layer
        
        # Return model output
        return logits


In [76]:
X_raw_train_npy_orig

array([[ 0.15693399, -0.19200465,  1.24225113, ..., -0.52726602,
         1.50944471,  1.94603694],
       [-0.28760006,  1.33413896,  2.61789836, ...,  0.16872513,
         3.50191173, -0.57236381],
       [-0.95440114,  0.13828016, -0.68365498, ..., -0.52726602,
        -0.48302231,  0.68683657],
       ...,
       [ 0.15693399, -0.86775978, -0.40852554, ...,  0.16872513,
        -0.48302231, -0.57236381],
       [ 0.2013874 , -0.74247933, -0.40852554, ...,  2.95268969,
        -0.48302231, -0.57236381],
       [-0.37650687, -0.25654306,  3.58085142, ...,  0.16872513,
        -0.48302231, -0.57236381]])

In [77]:
import random
random.seed(3)
torch.manual_seed(0)

In [78]:
testkfold=[]
nfold=5
kfold = KFold(nfold,  True )
for train, test in kfold.split(X_raw_train_npy_orig):
#	print('train: %s, test: %s' % (X_raw_train_npy_orig[train].shape, X_raw_train_npy_orig[test].shape))
    X_raw_train=X_raw_train_npy_orig[train]
    X_raw_test=X_raw_train_npy_orig[test]
    y_train=np.array(y_train_npy_orig[train] , dtype=float)
    y_test=np.array(y_train_npy_orig[test], dtype=float)
    
    X_raw_train = torch.from_numpy(X_raw_train) #format torch data
    y_train = torch.from_numpy(y_train)
    X_raw_test = torch.from_numpy(X_raw_test)
    y_test = torch.from_numpy(y_test)

    X_raw_train = X_raw_train.float() #float
    X_raw_test = X_raw_test.float()
    
    #X_train = X_raw_train / 255.0 #scale
    #X_test = X_raw_test / 255.0
    X_train = X_raw_train 
    X_test = X_raw_test
    
    train_dataset = MNISTDataset(X_train, y_train) #pytorch data
    test_dataset = MNISTDataset(X_test, y_test)
    
    batch_size = 10
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
        # Define model input and output dimensions from data
    input_dim = X_train.shape[1]                  # second dimension is data dimensionality
    output_dim = len(np.unique(y_train.numpy()))  # output dim. is equal to number of unique labels

    # Instantiate multi-layer perceptron model
    mlp_model = MLP(input_dim=input_dim, output_dim=output_dim)

    # Instantiate stochastic gradient descent (SGD) optimizer
    lr = 0.1
    #optimizer = torch.optim.SGD(mlp_model.parameters(), lr=lr)
    optimizer = torch.optim.Adam(mlp_model.parameters())

    
    print('\nTraining logistic regression model...\n')# Time model training
    time_start = time.time()

    # Place model in training mode
    # .train() method affects operations such as dropout and batch normalization
    mlp_model.train()

    # Train model/iterate over epochs
    for epoch in range(n_epochs):
        # Initialize epoch metrics variables
        n_obs = 0
        loss_sum = 0
        n_correct = 0

        # Iterate through training data mini-batches
        for images_batch, labels_batch in train_loader:
            # Zero out the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = mlp_model(images_batch)         # model predictions
            loss = loss_func(y_pred, labels_batch.long())  # loss function evaluation

            # Backward pass
            loss.backward()    # backpropagation
            optimizer.step()   # update parameters according to learning rate, gradients

            # Update epoch metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            loss_sum += n_batch * loss.item()
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

        # Calculate epoch training loss and training accuracy
        loss_train = loss_sum / n_obs
        acc_train = n_correct / n_obs

        # Display training progress
        prog_disp_freq = 1   # frequency of training progress display
        if (epoch + 1) % prog_disp_freq == 0 or epoch == 0 or epoch + 1 == n_epochs:
            print('  E%02d | train loss: %s | train acc.: %s' % 
                  (epoch + 1, '{:.4f}'.format(loss_train), '{:.4f}'.format(acc_train)))

    # Time model training
    time_end = time.time()
    print('\nMLP model training complete.\n')
    print('Time to train MLP model:  %.1f s\n' % (time_end - time_start))
    # Place model in evaluation mode
    # .eval() method affects operations such as dropout and batch normalization
    mlp_model.eval()

    # Initialize test set metrics variables
    n_obs = 0
    n_correct = 0

    # Disable gradient calculation
    with torch.no_grad():
        # Iterate through test data mini-batches
        for images_batch, labels_batch in test_loader:
            # Forward pass
            y_pred = mlp_model(images_batch)  # model predictions

            # Update test set metrics variables
            n_batch = len(labels_batch)
            n_obs += n_batch
            n_correct += (y_pred.argmax(1) == labels_batch.long()).float().sum().item()

    # Calculate test accuracy
    acc_test = n_correct / n_obs

    # Display test accuracy
    print('\nMLP model MNIST test acc.:  %.4f\n' % (acc_test))
    testkfold.append(acc_test)



Training logistic regression model...

  E01 | train loss: 0.5397 | train acc.: 0.7308
  E02 | train loss: 0.0182 | train acc.: 1.0000
  E03 | train loss: 0.0056 | train acc.: 1.0000
  E04 | train loss: 0.0001 | train acc.: 1.0000
  E05 | train loss: 0.0000 | train acc.: 1.0000
  E06 | train loss: 0.0000 | train acc.: 1.0000
  E07 | train loss: 0.0000 | train acc.: 1.0000
  E08 | train loss: 0.0000 | train acc.: 1.0000
  E09 | train loss: 0.0000 | train acc.: 1.0000
  E10 | train loss: 0.0000 | train acc.: 1.0000
  E11 | train loss: 0.0000 | train acc.: 1.0000
  E12 | train loss: 0.0000 | train acc.: 1.0000
  E13 | train loss: 0.0000 | train acc.: 1.0000
  E14 | train loss: 0.0000 | train acc.: 1.0000
  E15 | train loss: 0.0000 | train acc.: 1.0000
  E16 | train loss: 0.0000 | train acc.: 1.0000
  E17 | train loss: 0.0000 | train acc.: 1.0000
  E18 | train loss: 0.0000 | train acc.: 1.0000
  E19 | train loss: 0.0000 | train acc.: 1.0000
  E20 | train loss: 0.0000 | train acc.: 1.0000


  E68 | train loss: 0.0000 | train acc.: 1.0000
  E69 | train loss: 0.0000 | train acc.: 1.0000
  E70 | train loss: 0.0000 | train acc.: 1.0000
  E71 | train loss: 0.0000 | train acc.: 1.0000
  E72 | train loss: 0.0000 | train acc.: 1.0000
  E73 | train loss: 0.0000 | train acc.: 1.0000
  E74 | train loss: 0.0000 | train acc.: 1.0000
  E75 | train loss: 0.0000 | train acc.: 1.0000
  E76 | train loss: 0.0000 | train acc.: 1.0000
  E77 | train loss: 0.0000 | train acc.: 1.0000
  E78 | train loss: 0.0000 | train acc.: 1.0000
  E79 | train loss: 0.0000 | train acc.: 1.0000
  E80 | train loss: 0.0000 | train acc.: 1.0000
  E81 | train loss: 0.0000 | train acc.: 1.0000
  E82 | train loss: 0.0000 | train acc.: 1.0000
  E83 | train loss: 0.0000 | train acc.: 1.0000
  E84 | train loss: 0.0000 | train acc.: 1.0000
  E85 | train loss: 0.0000 | train acc.: 1.0000
  E86 | train loss: 0.0000 | train acc.: 1.0000
  E87 | train loss: 0.0000 | train acc.: 1.0000
  E88 | train loss: 0.0000 | train acc.:

  E33 | train loss: 0.0000 | train acc.: 1.0000
  E34 | train loss: 0.0000 | train acc.: 1.0000
  E35 | train loss: 0.0000 | train acc.: 1.0000
  E36 | train loss: 0.0000 | train acc.: 1.0000
  E37 | train loss: 0.0000 | train acc.: 1.0000
  E38 | train loss: 0.0000 | train acc.: 1.0000
  E39 | train loss: 0.0000 | train acc.: 1.0000
  E40 | train loss: 0.0000 | train acc.: 1.0000
  E41 | train loss: 0.0000 | train acc.: 1.0000
  E42 | train loss: 0.0000 | train acc.: 1.0000
  E43 | train loss: 0.0000 | train acc.: 1.0000
  E44 | train loss: 0.0000 | train acc.: 1.0000
  E45 | train loss: 0.0000 | train acc.: 1.0000
  E46 | train loss: 0.0000 | train acc.: 1.0000
  E47 | train loss: 0.0000 | train acc.: 1.0000
  E48 | train loss: 0.0000 | train acc.: 1.0000
  E49 | train loss: 0.0000 | train acc.: 1.0000
  E50 | train loss: 0.0000 | train acc.: 1.0000
  E51 | train loss: 0.0000 | train acc.: 1.0000
  E52 | train loss: 0.0000 | train acc.: 1.0000
  E53 | train loss: 0.0000 | train acc.:

In [79]:
np.mean(testkfold)

0.7761904761904762

In [80]:
testkfold

[0.7142857142857143, 1.0, 1.0, 0.6666666666666666, 0.5]

In [81]:
X_raw_test

tensor([[ 0.1569, -0.1920,  1.2423,  ..., -0.5273,  1.5094,  1.9460],
        [ 0.2236,  0.4040, -0.4085,  ..., -0.5273,  1.5094,  1.9460],
        [-0.5543, -0.1351, -0.1334,  ...,  0.8647, -0.4830, -0.5724],
        [ 0.2236,  2.1769,  0.4169,  ..., -0.5273, -0.4830, -0.5724],
        [ 0.2458,  0.1003, -0.4085,  ...,  0.1687, -0.4830, -0.5724],
        [ 0.2014, -0.7425, -0.4085,  ...,  2.9527, -0.4830, -0.5724]])